In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [2]:
csv_path = os.path.join("..", "..", "data","LINKED_DATA", "TSR_EHR", "TSR_3_CLEANED.csv")
tsr_3 = pd.read_csv(csv_path)
tsr_3.head()

,height_nm,weight_nm,edu_id,pro_id,opc_id,ih_fl,ivtpamg_nm,hospitalised_time,nivtpa_id,nivtpa1_fl,...,nihs_7_out,nihs_8_out,nihs_9_out,nihs_10_out,nihs_11_out,total_out,SexName,Age,mrs_tx_1,mrs_tx_3
0,153.0,62.0,3,1,3,0,0.0,8.0,0,999,...,1,1,0,1,0,4,0,67.0,1,1
1,152.0,62.0,3,1,2,0,0.0,4.0,0,999,...,0,1,0,0,0,1,0,69.0,1,0
2,148.0,56.0,2,999,2,0,0.0,5.0,0,999,...,0,1,0,0,0,2,0,71.0,0,0
3,152.0,56.0,4,1,2,0,0.0,3.0,1,0,...,0,0,0,0,0,0,0,71.0,0,0
4,160.0,60.0,2,1,3,0,0.0,4.0,0,999,...,0,0,0,0,0,4,0,62.0,3,3


In [3]:
tsr_3_input = tsr_3.drop(["mrs_tx_3"], axis=1)
tsr_3_input[tsr_3_input == "N"] = 0
tsr_3_input[tsr_3_input == "Y"] = 1
tsr_3_input = tsr_3_input.astype("float64")
tsr_3_input = np.array(tsr_3_input.values)

tsr_3_input_nomrs = tsr_3.drop(["mrs_tx_3", "mrs_tx_1"], axis=1)
tsr_3_input_nomrs[tsr_3_input_nomrs == "N"] = 0
tsr_3_input_nomrs[tsr_3_input_nomrs == "Y"] = 1
tsr_3_input_nomrs = tsr_3_input_nomrs.astype("float64")
tsr_3_input_nomrs = np.array(tsr_3_input_nomrs.values)

# 6 classes

In [4]:
tsr_3_output = tsr_3.mrs_tx_1
tsr_3_output = tsr_3_output.astype("float64")
tsr_3_output = np.array(tsr_3_output.values)

## SVM

In [5]:
svr = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores = cross_val_score(svr,tsr_3_input,tsr_3_output,cv = 10, scoring='r2')
print(svr_scores)
print("Mean of R^2:", svr_scores.mean())
print("Std of R^2:", svr_scores.std())

[0.99694273 0.98606705 0.97507974 0.99001265 0.97946683 0.97822353
 0.93773754 0.93069012 0.99708053 0.91708152]
Mean of R^2: 0.968838223524956
Std of R^2: 0.027701938328636384


In [6]:
svr.fit(tsr_3_input,tsr_3_output)
svr_predict =svr.predict(tsr_3_input)
print(svr_predict)
svr_predict = np.round(svr_predict)
print(svr_predict)
print("Accuracy of RF:", (tsr_3_output == svr_predict).sum() / len(svr_predict))

[1.04877115 1.04908655 0.06418789 ... 5.0157436  5.04873924 4.8565692 ]
[1. 1. 0. ... 5. 5. 5.]
Accuracy of RF: 0.9995790359924226


In [7]:
svr_pred = cross_val_predict(svr,tsr_3_input,tsr_3_output,cv = 10)
svr_pred = np.round(svr_pred)
confusion_matrix(tsr_3_output, svr_pred)

array([[   0,    0,    0,    0,    0,    0,    0],
       [   2,  424,   70,    4,    0,    0,    0],
       [   0,    0, 1086,   54,    4,    0,    0],
       [   0,    0,   23,  727,   21,    0,    0],
       [   0,    0,    0,   29,  628,   10,    0],
       [   0,    0,    0,    4,   48,  900,    7],
       [   0,    0,    0,    0,    5,   44,  661]], dtype=int64)

In [8]:
svr_pred[svr_pred == -1] = 0
confusion_matrix(tsr_3_output, svr_pred)

array([[ 426,   70,    4,    0,    0,    0],
       [   0, 1086,   54,    4,    0,    0],
       [   0,   23,  727,   21,    0,    0],
       [   0,    0,   29,  628,   10,    0],
       [   0,    0,    4,   48,  900,    7],
       [   0,    0,    0,    5,   44,  661]], dtype=int64)

In [9]:
svr1 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores1 = cross_val_score(svr1,tsr_3_input_nomrs,tsr_3_output,cv = 10, scoring='r2')
print(svr_scores1)
print("Mean of R^2:", svr_scores1.mean())
print("Std of R^2:", svr_scores1.std())

[0.75387908 0.76861388 0.83189696 0.80334807 0.82178857 0.81403305
 0.80603646 0.7588735  0.84258892 0.66742796]
Mean of R^2: 0.7868486450762257
Std of R^2: 0.04919342478037698


In [10]:
svr1.fit(tsr_3_input_nomrs,tsr_3_output)
svr_predict1 =svr1.predict(tsr_3_input_nomrs)
print(svr_predict1)
svr_predict1 = np.round(svr_predict1)
print(svr_predict1)
print("Accuracy of RF:", (tsr_3_output == svr_predict1).sum() / len(svr_predict1))

[1.17194557 0.90045753 0.82003352 ... 5.27031679 4.3583525  4.03077705]
[1. 1. 1. ... 5. 4. 4.]
Accuracy of RF: 0.5647232161650179


In [11]:
svr_pred1 = cross_val_predict(svr1,tsr_3_input_nomrs,tsr_3_output,cv = 10)
svr_pred1 = np.round(svr_pred1)
confusion_matrix(tsr_3_output, svr_pred1)

array([[  0,   0,   0,   0,   0,   0,   0,   0],
       [  2,  34, 391,  61,  11,   0,   1,   0],
       [  0,  10, 868, 221,  35,   8,   2,   0],
       [  1,   1, 282, 377,  96,  14,   0,   0],
       [  0,   0,  21, 203, 369,  68,   6,   0],
       [  0,   0,   8,  30, 245, 545, 131,   0],
       [  0,   2,   4,   4,  26, 187, 469,  18],
       [  0,   0,   0,   0,   0,   0,   0,   0]], dtype=int64)

In [12]:
svr_pred1[svr_pred1 == -1] = 0
svr_pred1[svr_pred1 == 6] = 5
confusion_matrix(tsr_3_output, svr_pred1)

array([[ 36, 391,  61,  11,   0,   1],
       [ 10, 868, 221,  35,   8,   2],
       [  2, 282, 377,  96,  14,   0],
       [  0,  21, 203, 369,  68,   6],
       [  0,   8,  30, 245, 545, 131],
       [  2,   4,   4,  26, 187, 487]], dtype=int64)

## RF

In [13]:
rfr = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores = cross_val_score(rfr,tsr_3_input,tsr_3_output,cv = 10, scoring='r2')
print(rfr_scores)
print("Mean of R^2:", rfr_scores.mean())
print("Std of R^2:", rfr_scores.std())

[0.9999604  0.99997409 0.99998647 0.99999664 0.99998574 0.99998577
 0.9999849  0.99998505 0.99999245 0.99997482]
Mean of R^2: 0.9999826326079451
Std of R^2: 9.827182229605757e-06


In [14]:
rfr.fit(tsr_3_input,tsr_3_output)
rfr_predict =rfr.predict(tsr_3_input)
print(rfr_predict)
rfr_predict = np.round(rfr_predict)
print(rfr_predict)
print("Accuracy of RF:", (tsr_3_output == rfr_predict).sum() / len(rfr_predict))

[1. 1. 0. ... 5. 5. 5.]
[1. 1. 0. ... 5. 5. 5.]
Accuracy of RF: 1.0


In [15]:
rfr_pred = cross_val_predict(rfr,tsr_3_input,tsr_3_output,cv = 10)
rfr_pred = np.round(rfr_pred)
confusion_matrix(tsr_3_output, rfr_pred)

array([[ 500,    0,    0,    0,    0,    0],
       [   0, 1144,    0,    0,    0,    0],
       [   0,    0,  771,    0,    0,    0],
       [   0,    0,    0,  667,    0,    0],
       [   0,    0,    0,    0,  959,    0],
       [   0,    0,    0,    0,    0,  710]], dtype=int64)

In [16]:
rfr1 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores1 = cross_val_score(rfr1,tsr_3_input_nomrs,tsr_3_output,cv = 10, scoring='r2')
print(rfr_scores1)
print("Mean of R^2:", rfr_scores1.mean())
print("Std of R^2:", rfr_scores1.std())

[0.79848646 0.83124349 0.86398329 0.8160926  0.84065532 0.85591027
 0.8449297  0.7813848  0.89386159 0.81143728]
Mean of R^2: 0.8337984798534303
Std of R^2: 0.03161128500485248


In [17]:
rfr1.fit(tsr_3_input_nomrs,tsr_3_output)
rfr_predict1 =rfr1.predict(tsr_3_input_nomrs)
print(rfr_predict1)
rfr_predict1 = np.round(rfr_predict1)
print(rfr_predict1)
print("Accuracy of RF:", (tsr_3_output == rfr_predict1).sum() / len(rfr_predict1))

[0.93333333 1.         0.         ... 5.         4.93333333 4.6       ]
[1. 1. 0. ... 5. 5. 5.]
Accuracy of RF: 0.9379078088823406


In [18]:
rfr_pred1 = cross_val_predict(rfr1,tsr_3_input_nomrs,tsr_3_output,cv = 10)
rfr_pred1 = np.round(rfr_pred1)
confusion_matrix(tsr_3_output, rfr_pred1)

array([[296, 160,  31,  11,   1,   1],
       [ 76, 779, 235,  42,  10,   2],
       [  2, 125, 505, 118,  21,   0],
       [  0,  21, 125, 431,  87,   3],
       [  0,   9,  16, 167, 720,  47],
       [  0,   2,   4,  16, 178, 510]], dtype=int64)

## XGBoost

In [19]:
xgbr = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores = cross_val_score(xgbr,tsr_3_input,tsr_3_output,cv = 10, scoring='r2')
print(xgbr_scores)
print("Mean of R^2:", xgbr_scores.mean())
print("Std of R^2:", xgbr_scores.std())

[0.99993267 0.99994881 0.99994648 0.99995066 0.99994196 0.99994731
 0.99994212 0.99993815 0.99993076 0.99991742]
Mean of R^2: 0.9999396339808422
Std of R^2: 9.726589976168125e-06


In [20]:
xgbr.fit(tsr_3_input,tsr_3_output)
xgbr_predict =xgbr.predict(tsr_3_input)
print(xgbr_predict)
xgbr_predict = np.round(xgbr_predict)
print(xgbr_predict)
print("Accuracy of RF:", (tsr_3_output == xgbr_predict).sum() / len(xgbr_predict))

[9.9761289e-01 9.9761289e-01 2.4044225e-03 ... 4.9784422e+00 4.9784422e+00
 4.9784422e+00]
[1. 1. 0. ... 5. 5. 5.]
Accuracy of RF: 1.0


In [21]:
xgbr_pred = cross_val_predict(xgbr,tsr_3_input,tsr_3_output,cv = 10)
xgbr_pred = np.round(xgbr_pred)
confusion_matrix(tsr_3_output, xgbr_pred)

array([[ 500,    0,    0,    0,    0,    0],
       [   0, 1144,    0,    0,    0,    0],
       [   0,    0,  771,    0,    0,    0],
       [   0,    0,    0,  667,    0,    0],
       [   0,    0,    0,    0,  959,    0],
       [   0,    0,    0,    0,    0,  710]], dtype=int64)

In [22]:
xgbr1 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
xgbr_scores1 = cross_val_score(xgbr1,tsr_3_input_nomrs,tsr_3_output,cv = 10, scoring='r2')
print(xgbr_scores1)
print("Mean of R^2:", xgbr_scores1.mean())
print("Std of R^2:", xgbr_scores1.std())

[0.79848646 0.83124349 0.86398329 0.8160926  0.84065532 0.85591027
 0.8449297  0.7813848  0.89386159 0.81143728]
Mean of R^2: 0.8337984798534303
Std of R^2: 0.03161128500485248


In [23]:
xgbr1.fit(tsr_3_input_nomrs,tsr_3_output)
xgbr_predict1 =xgbr1.predict(tsr_3_input_nomrs)
print(xgbr_predict1)
xgbr_predict1 = np.round(xgbr_predict1)
print(xgbr_predict1)
print("Accuracy of RF:", (tsr_3_output == xgbr_predict1).sum() / len(xgbr_predict1))

[0.93333333 1.         0.         ... 5.         4.93333333 4.6       ]
[1. 1. 0. ... 5. 5. 5.]
Accuracy of RF: 0.9379078088823406


In [24]:
xgbr_pred1 = cross_val_predict(xgbr1,tsr_3_input_nomrs,tsr_3_output,cv = 10)
xgbr_pred1 = np.round(xgbr_pred1)
confusion_matrix(tsr_3_output, xgbr_pred1)

array([[296, 160,  31,  11,   1,   1],
       [ 76, 779, 235,  42,  10,   2],
       [  2, 125, 505, 118,  21,   0],
       [  0,  21, 125, 431,  87,   3],
       [  0,   9,  16, 167, 720,  47],
       [  0,   2,   4,  16, 178, 510]], dtype=int64)

# 2 classes

In [25]:
tsr_3_output[(tsr_3_output == 0)|(tsr_3_output == 1)|(tsr_3_output == 2)] = 0
tsr_3_output[(tsr_3_output == 3)|(tsr_3_output == 4)|(tsr_3_output == 5)] = 1

## SVM

In [26]:
svr2 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores2 = cross_val_score(svr2,tsr_3_input,tsr_3_output,cv = 10, scoring='r2')
print(svr_scores2)
print("Mean of R^2:", svr_scores2.mean())
print("Std of R^2:", svr_scores2.std())

[0.75140732 0.80245764 0.81446121 0.85211964 0.8428527  0.80778785
 0.80576081 0.81369901 0.82505254 0.72815394]
Mean of R^2: 0.8043752659309547
Std of R^2: 0.03606769245824599


In [27]:
svr2.fit(tsr_3_input,tsr_3_output)
svr_predict2 =svr2.predict(tsr_3_input)
print(svr_predict2)
svr_predict2 = np.round(svr_predict2)
print(svr_predict2)
print("Accuracy of RF:", (tsr_3_output == svr_predict2).sum() / len(svr_predict2))

[ 0.00911766  0.01407269 -0.1753291  ...  0.97216899  1.22712792
  1.19348218]
[ 0.  0. -0. ...  1.  1.  1.]
Accuracy of RF: 0.978530835613555


In [28]:
svr_pred2 = cross_val_predict(svr2,tsr_3_input,tsr_3_output,cv = 10)
svr_pred2 = np.round(svr_pred2)
confusion_matrix(tsr_3_output, svr_pred2)

array([[2341,   74],
       [  76, 2260]], dtype=int64)

In [29]:
svr3 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores3 = cross_val_score(svr3,tsr_3_input_nomrs,tsr_3_output,cv = 10, scoring='r2')
print(svr_scores3)
print("Mean of R^2:", svr_scores3.mean())
print("Std of R^2:", svr_scores3.std())

[0.63299409 0.67364712 0.73980537 0.730141   0.75369505 0.70540712
 0.71956343 0.67518402 0.77269368 0.53585951]
Mean of R^2: 0.6938990384976448
Std of R^2: 0.06591086758052825


In [30]:
svr3.fit(tsr_3_input_nomrs,tsr_3_output)
svr_predict3 =svr3.predict(tsr_3_input_nomrs)
print(svr_predict3)
svr_predict3 = np.round(svr_predict3)
print(svr_predict3)
print("Accuracy of RF:", (tsr_3_output == svr_predict3).sum() / len(svr_predict3))

[ 0.02920437  0.01130879 -0.01810003 ...  1.03223808  1.10867321
  1.02422355]
[ 0.  0. -0. ...  1.  1.  1.]
Accuracy of RF: 0.9179120185224163


In [31]:
svr_pred3 = cross_val_predict(svr3,tsr_3_input_nomrs,tsr_3_output,cv = 10)
svr_pred3 = np.round(svr_pred3)
confusion_matrix(tsr_3_output, svr_pred3)

array([[2219,  196],
       [ 238, 2098]], dtype=int64)

## RF

In [32]:
rfr2 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores2 = cross_val_score(rfr2,tsr_3_input,tsr_3_output,cv = 10, scoring='r2')
print(rfr_scores2)
print("Mean of R^2:", rfr_scores2.mean())
print("Std of R^2:", rfr_scores2.std())

[1.         1.         1.         0.99996156 1.         1.
 1.         1.         1.         1.        ]
Mean of R^2: 0.9999961563048739
Std of R^2: 1.1531085378579942e-05


In [33]:
rfr2.fit(tsr_3_input,tsr_3_output)
rfr_predict2 =rfr2.predict(tsr_3_input)
print(rfr_predict2)
rfr_predict2 = np.round(rfr_predict2)
print(rfr_predict2)
print("Accuracy of RF:", (tsr_3_output == rfr_predict2).sum() / len(rfr_predict2))

[0. 0. 0. ... 1. 1. 1.]
[0. 0. 0. ... 1. 1. 1.]
Accuracy of RF: 1.0


In [34]:
rf_pred2 = cross_val_predict(rfr2,tsr_3_input,tsr_3_output,cv = 10)
rf_pred2 = np.round(rf_pred2)
confusion_matrix(tsr_3_output, rf_pred2)

array([[2415,    0],
       [   0, 2336]], dtype=int64)

In [35]:
rfr3 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores3 = cross_val_score(rfr3,tsr_3_input_nomrs,tsr_3_output,cv = 10, scoring='r2')
print(rfr_scores3)
print("Mean of R^2:", rfr_scores3.mean())
print("Std of R^2:", rfr_scores3.std())

[0.67450974 0.71671835 0.83315047 0.7694936  0.77265994 0.74529902
 0.75116166 0.77124322 0.84322987 0.62267818]
Mean of R^2: 0.7500144060955003
Std of R^2: 0.06323867093793825


In [36]:
rfr3.fit(tsr_3_input_nomrs,tsr_3_output)
rfr_predict3 =rfr3.predict(tsr_3_input_nomrs)
print(rfr_predict3)
rfr_predict3 = np.round(rfr_predict3)
print(rfr_predict3)
print("Accuracy of RF:", (tsr_3_output == rfr_predict3).sum() / len(rfr_predict3))

[0.06666667 0.         0.         ... 1.         1.         1.        ]
[0. 0. 0. ... 1. 1. 1.]
Accuracy of RF: 0.9972637339507472


In [37]:
rfr_pred3 = cross_val_predict(rfr3,tsr_3_input_nomrs,tsr_3_output,cv = 10)
rfr_pred3 = np.round(rfr_pred3)
confusion_matrix(tsr_3_output, rfr_pred3)

array([[2184,  231],
       [ 128, 2208]], dtype=int64)

## XGBoost

In [38]:
xgbr2 = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores2 = cross_val_score(xgbr2,tsr_3_input,tsr_3_output,cv = 10, scoring='r2')
print(xgbr_scores2)
print("Mean of R^2:", xgbr_scores2.mean())
print("Std of R^2:", xgbr_scores2.std())

[0.99997726 0.99997712 0.99997712 0.99997671 0.99997732 0.99997719
 0.99997694 0.9999773  0.99997729 0.99997552]
Mean of R^2: 0.999976976946966
Std of R^2: 5.183694326287556e-07


In [39]:
xgbr2.fit(tsr_3_input,tsr_3_output)
xgbr_predict2 =xgbr2.predict(tsr_3_input)
print(xgbr_predict2)
xgbr_predict2 = np.round(xgbr_predict2)
print(xgbr_predict2)
print("Accuracy of RF:", (tsr_3_output == xgbr_predict2).sum() / len(xgbr_predict2))

[0.0023801 0.0023801 0.0023801 ... 0.9976197 0.9976197 0.9976197]
[0. 0. 0. ... 1. 1. 1.]
Accuracy of RF: 1.0


In [40]:
xgbr_pred2 = cross_val_predict(xgbr2,tsr_3_input,tsr_3_output,cv = 10)
xgbr_pred2 = np.round(xgbr_pred2)
confusion_matrix(tsr_3_output, xgbr_pred2)

array([[2415,    0],
       [   0, 2336]], dtype=int64)

In [41]:
xgbr3 = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores3 = cross_val_score(xgbr3,tsr_3_input_nomrs,tsr_3_output,cv = 10, scoring='r2')
print(xgbr_scores3)
print("Mean of R^2:", xgbr_scores3.mean())
print("Std of R^2:", xgbr_scores3.std())

[0.71489409 0.74967602 0.84586303 0.76303676 0.75537171 0.7370609
 0.7565685  0.74812115 0.8265216  0.64069671]
Mean of R^2: 0.7537810472258741
Std of R^2: 0.05338066926479723


In [42]:
xgbr3.fit(tsr_3_input_nomrs,tsr_3_output)
xgbr_predict3 =xgbr3.predict(tsr_3_input_nomrs)
print(xgbr_predict3)
xgbr_predict3 = np.round(xgbr_predict3)
print(xgbr_predict3)
print("Accuracy of RF:", (tsr_3_output == xgbr_predict3).sum() / len(xgbr_predict3))

[ 0.01006533  0.01087    -0.00179522 ...  1.0187713   1.007954
  1.0018022 ]
[ 0.  0. -0. ...  1.  1.  1.]
Accuracy of RF: 0.9795832456324984


In [43]:
xgbr_pred3 = cross_val_predict(xgbr3,tsr_3_input_nomrs,tsr_3_output,cv = 10)
xgbr_pred3 = np.round(xgbr_pred3)
confusion_matrix(tsr_3_output, xgbr_pred3)

array([[2187,  228],
       [ 141, 2195]], dtype=int64)

# Summary

## Mean & Std

In [44]:
svr_mean = np.array([svr_scores.mean(), svr_scores.std(), svr_scores[0], svr_scores[1], svr_scores[2], svr_scores[3],
                     svr_scores[4], svr_scores[5], svr_scores[6], svr_scores[7], svr_scores[8], svr_scores[9]])
rfr_mean = np.array([rfr_scores.mean(), rfr_scores.std(), rfr_scores[0], rfr_scores[1], rfr_scores[2], rfr_scores[3],rfr_scores[4], 
                    rfr_scores[5], rfr_scores[6], rfr_scores[7], rfr_scores[8], rfr_scores[9]])
xgbr_mean = np.array([xgbr_scores.mean(), xgbr_scores.std(), xgbr_scores[0], xgbr_scores[1], xgbr_scores[2], xgbr_scores[3],
                     xgbr_scores[4], xgbr_scores[5], xgbr_scores[6], xgbr_scores[7], xgbr_scores[8], xgbr_scores[9]])
svr_mean2 = np.array([svr_scores2.mean(), svr_scores2.std(), svr_scores2[0], svr_scores2[1], svr_scores2[2], svr_scores2[3],
                     svr_scores2[4], svr_scores2[5], svr_scores2[6], svr_scores2[7], svr_scores2[8], svr_scores2[9]])
rfr_mean2 = np.array([rfr_scores2.mean(), rfr_scores2.std(), rfr_scores2[0], rfr_scores2[1], rfr_scores2[2], rfr_scores2[3],rfr_scores2[4], 
                    rfr_scores2[5], rfr_scores2[6], rfr_scores2[7], rfr_scores2[8], rfr_scores2[9]])
xgbr_mean2 = np.array([xgbr_scores2.mean(), xgbr_scores2.std(), xgbr_scores2[0], xgbr_scores2[1], xgbr_scores2[2], xgbr_scores2[3],
                     xgbr_scores2[4], xgbr_scores2[5], xgbr_scores2[6], xgbr_scores2[7], xgbr_scores2[8], xgbr_scores2[9]])

In [45]:
tsr_3_mean = pd.DataFrame([svr_mean, rfr_mean, xgbr_mean, svr_mean2, rfr_mean2, xgbr_mean2]).T
tsr_3_mean.index = ["Mean", "Std", "R^2_1", "R^2_2", "R^2_3", "R^2_4", "R^2_5", "R^2_6", "R^2_7", "R^2_8", "R^2_9", "R^2_10"]
tsr_3_mean.columns = ["svr", 'rfr', 'xgbr', 'svr2', 'rfr2', 'xgbr2']

In [46]:
csv_save = os.path.join(".", "tsr_3_mean_regression.csv")
tsr_3_mean.to_csv(csv_save, index = True)